In [ ]:
!pip install transformers --quiet

In [ ]:
!pip install bitsandbytes accelerate --quiet



In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model_name = "facebook/opt-1.3b"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
prompt = "India is one of the"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generated_ids = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=80)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# create textarea widget
text_area = widgets.Textarea(
    value='',
    placeholder='Type something',
    disabled=False
)

text_area.layout.height = "300px"
text_area.layout.width = "750px"

# create button widget
button = widgets.Button(description="Submit")

def generate(prompt):
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
  generated_ids = model.generate(input_ids, do_sample=True, temperature=0.9, max_new_tokens=80)
  generated_text = tokenizer.decode(generated_ids[0])
  return generated_text

def remove_end_tags(s):
    # check if string starts with "</s>"
    if s.startswith("</s>"):
        # remove "</s>" from the start of the string
        s = s[len("</s>"):]
    
    # check if string ends with "</s>"
    if s.endswith("</s>"):
        # remove "</s>" from the end of the string
        s = s[:-len("</s>")]
    
    # return the modified string
    return s

def on_submit(btn):
    btn.description = "loading..."
    text = text_area.value
    updated_text = generate(text)
    text_area.value = remove_end_tags(updated_text)
    btn.description = "Submit"

# attach the callback function to the button click event
button.on_click(on_submit)

# display the widgets
display(text_area, button)
